# Project 4 Notebook: NLP Classification

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join(os.pardir, os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import pandas as pd
import nltk
from sklearn.model_selection import train_test_split
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.tokenize import regexp_tokenize, word_tokenize, RegexpTokenizer
import matplotlib.pyplot as plt
import string
import re
from nltk.corpus import stopwords

In [2]:
#Creation of Dataframe

tweets = pd.read_csv('./data/tweets.csv', encoding='unicode_escape')
tweets.shape

(9093, 3)

In [3]:
tweets.head()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion


In [4]:
tweets.loc[0].tweet_text

'.@wesley83 I have a 3G iPhone. After 3 hrs tweeting at #RISE_Austin, it was dead!  I need to upgrade. Plugin stations at #SXSW.'

In [5]:
#Creation of list containing Classification

classif = ['Positive', 'Negative', 'Neutral']

In [6]:
#Creation of predictied classification column

tweets['predicted_classif'] = [[] for x in tweets['tweet_text']]

In [7]:
tweets.head()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product,predicted_classif
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion,[]
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion,[]
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion,[]
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion,[]
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion,[]


In [50]:
tweets.shape

(9092, 4)

## Data Cleaning

In [8]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9093 entries, 0 to 9092
Data columns (total 4 columns):
 #   Column                                              Non-Null Count  Dtype 
---  ------                                              --------------  ----- 
 0   tweet_text                                          9092 non-null   object
 1   emotion_in_tweet_is_directed_at                     3291 non-null   object
 2   is_there_an_emotion_directed_at_a_brand_or_product  9093 non-null   object
 3   predicted_classif                                   9093 non-null   object
dtypes: object(4)
memory usage: 284.3+ KB


In [9]:
tweets['emotion_in_tweet_is_directed_at'].fillna('Unknown', inplace=True)

In [10]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9093 entries, 0 to 9092
Data columns (total 4 columns):
 #   Column                                              Non-Null Count  Dtype 
---  ------                                              --------------  ----- 
 0   tweet_text                                          9092 non-null   object
 1   emotion_in_tweet_is_directed_at                     9093 non-null   object
 2   is_there_an_emotion_directed_at_a_brand_or_product  9093 non-null   object
 3   predicted_classif                                   9093 non-null   object
dtypes: object(4)
memory usage: 284.3+ KB


In [11]:
tweets.tail()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product,predicted_classif
9088,Ipad everywhere. #SXSW {link},iPad,Positive emotion,[]
9089,"Wave, buzz... RT @mention We interrupt your re...",Unknown,No emotion toward brand or product,[]
9090,"Google's Zeiger, a physician never reported po...",Unknown,No emotion toward brand or product,[]
9091,Some Verizon iPhone customers complained their...,Unknown,No emotion toward brand or product,[]
9092,Ï¡Ïàü_ÊÎÒ£Áââ_£â_ÛâRT @...,Unknown,No emotion toward brand or product,[]


In [12]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9093 entries, 0 to 9092
Data columns (total 4 columns):
 #   Column                                              Non-Null Count  Dtype 
---  ------                                              --------------  ----- 
 0   tweet_text                                          9092 non-null   object
 1   emotion_in_tweet_is_directed_at                     9093 non-null   object
 2   is_there_an_emotion_directed_at_a_brand_or_product  9093 non-null   object
 3   predicted_classif                                   9093 non-null   object
dtypes: object(4)
memory usage: 284.3+ KB


In [13]:
tweets = tweets[tweets['tweet_text'].notna()]

In [14]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9092 entries, 0 to 9092
Data columns (total 4 columns):
 #   Column                                              Non-Null Count  Dtype 
---  ------                                              --------------  ----- 
 0   tweet_text                                          9092 non-null   object
 1   emotion_in_tweet_is_directed_at                     9092 non-null   object
 2   is_there_an_emotion_directed_at_a_brand_or_product  9092 non-null   object
 3   predicted_classif                                   9092 non-null   object
dtypes: object(4)
memory usage: 355.2+ KB


In [51]:
tweets.shape

(9092, 4)

## Pre-processing

In [42]:
#Variable for string of tweet text

text = str(tweets["tweet_text"])

In [43]:
#text = tweets["tweet_text"].normalize('NFKD', doc).encode('ascii', 'ignore').decode('utf-8', 'ignore')

In [44]:
#Removing Link text

text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

In [45]:
#Removing @'s and hashtags'

text = re.sub(r'\@\w+|\#','', text)

In [46]:
#Regex Tokenizer

basic_token_pattern = r"(?u)\b\w\w+\b"

tokenizer = RegexpTokenizer(basic_token_pattern)
tokened_tweets = tokenizer.tokenize(text)

In [47]:
#Lowercasing tweets

lowered_tweets = [x.lower() for x in tokened_tweets]

In [48]:
#splitting tweets by spaces

split_tweets = [x.split(' ') for x in lowered_tweets]

In [56]:
tweets["text_tokenized"] = split_tweets

ValueError: Length of values (69) does not match length of index (9092)

In [ ]:
# finish prepreocessingbefore train test then vectorize

## Train-test

In [ ]:
X = split_tweets
y = tweets['is_there_an_emotion_directed_at_a_brand_or_product']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, train_size = 0.7, random_state=42)

In [ ]:
X_train.head()

In [ ]:
y_train.head()

In [ ]:
y_train.value_counts()

In [ ]:
tweets.info()

In [ ]:
#tweets['tweet_text'] = tweets['tweet_text'].apply(lambda x: " ".join(x.lower() for x in str(x).split() \
                                    #if x not in stop_words))